# <div style="text-align: center">18.335/6.337 Problem Set 1 - Floating Point and Norms</div>
## <div style="text-align: center">Solution</div>
### <div style="text-align: center">Created by Wonseok Shin</div>

$\newcommand{\Cmat}[2]{\mathbb{C}^{#1\times#2}}\newcommand{\Cvec}[1]{\mathbb{C}^{#1}}\newcommand{\Rmat}[2]{\mathbb{R}^{#1\times#2}}\newcommand{\Rvec}[1]{\mathbb{R}^{#1}}\newcommand{\null}{\mathrm{null}}\newcommand{\range}{\mathrm{range}}\newcommand{\rank}{\mathrm{rank}}\newcommand{\nullity}{\mathrm{nullity}}\newcommand{\sign}{\mathrm{sign}}\newcommand{\norm}[1]{\left\|#1\right\|}\newcommand{\abs}[1]{\left|#1\right|}\newcommand{\epsmach}{\epsilon_\mathrm{machine}}\newcommand{\log}{\mathrm{log}}\newcommand{\tanh}{\mathrm{tanh}}\newcommand{\l}{\lambda}\newcommand{\d}{\delta}$

## Problem 1.  Floating-point peculiarity [4 pts]

In this problem, we will see some peculiar behaviors of floating-point numbers and try to understand them.

A few things to keep in mind:

- As discussed in class, the bit representation of IEEE double-precision floating point numbers is $S\,E_1 \cdots E_{11} M_1 \cdots M_{52}$, where
    - a single bit $S$ stores the sign,
    - 11 bits $E_1 \cdots E_{11}$ store the exponent information, and 
    - 52 bits $M_1 \cdots M_{52}$ store the mantissa (or significand) information.
- `eps()` in Julia returns the gap between 1 and the next larger number in the double-precision floating-point system.  
    - This is *not* the "machine epsilon" $\epsmach = \frac{1}{2}\beta^{-(t-1)}$ defined in Eq. (13.3) of T&B, but is twice as big, i.e., `eps()` $=2\epsmach$.
    - T&B's definition of machine epsilon is not universal: many other authors take the value of `eps()` itself (not a half) as machine epsilon, as shown [here](https://en.wikipedia.org/wiki/Machine_epsilon#Values_for_standard_hardware_floating_point_arithmetics).
- `bin()` in Julia is useful for obtaining the binary representation of integers.  For example, `bin(23)` returns `"10111"`.

(a) [0.5 pts] Find the bit representation of the value returned by `eps()`.  (Indicate the values of the sign bit $S$, exponent bits $E_1 \cdots E_{11}$, and the mantissa bits $M_1 \cdots M_{52}$.)

***Solution.***

`eps()` is $(-1)^0 \times 1.0 \cdots 0_{(2)} \times 2^{-52} = (-1)^s m \beta^e$.  Therefore, 
- $S = 0$, 
- $M_1 \cdots M_{52} = 0 \cdots 0$, and 
- $E_1 \cdots E_{11}$ is the binary representation of $-52 + 1023$, which is $01111001011$ as shown below.

In [1]:
bin(-52+1023)

"1111001011"

Therefore, the complete bit representation for `eps()` is $SE_1 \cdots E_{11}M_1 \cdots M_{52} = 0011110010110000000000000000000000000000000000000000000000000000$.

(b) [1 pt] Evaluate `1+eps()-1` and `2+eps()-2`.  Explain why you get different results.  (Hint: fundamental axiom of floating-point arithmetic.)

***Solution.***

The operations used in these evaluations is a floating point operation.  According to the fundamental axiom of floating-point arithmetic,

`1+eps()` = $1 \oplus 2\epsmach = fl(1+2\epsmach) = fl({1.\overbrace{0 \cdots 0 1}^{\text{52 bits}}}_{(2)} \times 2^0) = {1.\overbrace{0 \cdots 0 1}^{\text{52 bits}}}_{(2)} \times 2^0 = 1 + 2\epsmach$

and

`2+eps()` = $2 \oplus 2\epsmach = fl(2+2\epsmach) = fl(2(1+\epsmach)) = fl(2 \times ({1.\overbrace{0 \cdots 0 1}^{\text{53 bits}}}_{(2)} \times 2^0)) = fl({1.\overbrace{0 \cdots 0 1}^{\text{53 bits}}}_{(2)} \times 2^1) = {1.\overbrace{0 \cdots 0}^{\text{52 bits}}}_{(2)} \times 2^1 = 2$.

In other words, `1+eps()` will remain exact whereas `2+eps()` will be approximated by 2.  Therefore `1+eps()-1` returns `eps()` and `2+eps()-2` returs 0.

(c) [0.5 pts] Evaluate `(2^-537.)^2` and `(2^-538.)^2`.  Explain why you get 0 from the latter.

***Solution.***

In exact arithmetic, `(2^-537.)^2` and `(2^-538.)^2` are $2^{-1074}$ and $2^{-1076}$, respectively.  The former is the smallest denormalized number of the double-precision floating-point format, whereas the latter is smaller than that.  Therefore, the latter suffers from underflow and is approximated by 0.

(d) [1 pt] (Taken from Exercise 13.2b of T&B)  The double-precision floating-point system includes many integers, but not all of them.  What is the smallest positive integer $n$ that does not belong to this system?  

***Solution.***

The double-precision floating-point system has 52 mantissa bits below the radix point.  The positive integers that do not belong to the system are therefore the ones that require more than the 52 mantissa bits.  The smallest such mantissa is ${1.\overbrace{0\cdots 01}^{\text{53 bits}}}_{(2)}$, and $2^{53}$ is the smallest power of two to multiply to this mantissa in order to obtain an integer.  Therefore, the number we are looking for is ${1.\overbrace{0\cdots 01}^{\text{53 bits}}}_{(2)} \times 2^{53} = 2^{53} + 1$.

(e) [1 pt] (Taken from Exercise 13.2c of T&B)  For the answer $n$ of Part (d), detertmine whether or not each of $n+1$, $n+2$, $n+3$ belongs to the double-precision floating-point system.

Write Julia code that validates your analysis.  Write the code such that it also confirms $n$ indeed does not belong to the double-precision floating-point system, whereas $n-3$, $n-2$, $n-1$ do (as $n$ is the smallest that does not).  (Hint: `float(n)` produces the floating-point approximation of an integer `n`.)

***Solution.***

For $n = 2^{53} + 1 = {1\overbrace{0\cdots 01}^{\text{53 bits}}}_{(2)} = {1.\overbrace{0\cdots 01}^{\text{53 bits}}}_{(2)}\times 2^{53}$,

- $n+1 = {1\overbrace{0\cdots 10}^{\text{53 bits}}}_{(2)} = {1.\overbrace{0\cdots 01}^{\text{52 bits}}}_{(2)} \times 2^{53}$ requires 52 mantissa bits,
- $n+2 = {1\overbrace{0\cdots 11}^{\text{53 bits}}}_{(2)} = {1.\overbrace{0\cdots 11}^{\text{53 bits}}}_{(2)} \times 2^{53}$ requires 53 mantissa bits, and
- $n+3 = {1\overbrace{0\cdots 100}^{\text{53 bits}}}_{(2)} = {1.\overbrace{0\cdots 01}^{\text{51 bits}}}_{(2)} \times 2^{53}$ requires 51 mantissa bits.

Therefore, $n+1$ and $n+3$ fit within the 52 mantissa bits of the IEEE double-precision floating-point format, but $n+2$ does not.

The following code validates this analysis, and also confirms that $n-3$, $n-2$, $n-1$ indeed belong to the double-precision floating-point system.

In [2]:
function test_n(n)
    println("n-3 == float(n-3)?  $(n-3==float(n-3))")
    println("n-2 == float(n-2)?  $(n-2==float(n-2))")
    println("n-1 == float(n-1)?  $(n-1==float(n-1))")    
    println("  n == float(n)?    $(n==float(n))")
    println("n+1 == float(n+1)?  $(n+1==float(n+1))")
    println("n+2 == float(n+2)?  $(n+2==float(n+2))")
    println("n+3 == float(n+3)?  $(n+3==float(n+3))")
end

test_n(2^53+1)

n-3 == float(n-3)?  true
n-2 == float(n-2)?  true
n-1 == float(n-1)?  true
  n == float(n)?    false
n+1 == float(n+1)?  true
n+2 == float(n+2)?  false
n+3 == float(n+3)?  true


## Problem 2.  Accurate evaluation of mathematical functions [3 pts]

In this problem, we will devise ways to evaluate some mathematical functions on computers accurately.

(a) [1 pt] Evaluation of $f(x) = e^x - 1$ around $x = 0$.

- Explain the numerical problem you may experience during this evaluation.
- Devise a method to avoid this problem.  (Hint: $\tanh(x)$.)
- Implement your method in `f_new(x)` below.

***Solution.***

For $x \approx 0$, $e^x \approx 1$, so subtracting $1$ from $e^x$ suffers from catastrophic cancellation.

To avoid this problem, consider $\tanh(x) = (e^x - e^{-x}) / (e^x + e^{-x}) = (e^{2x}-1) / (e^{2x}+1)$.  From this we can write $\tanh(x/2) = (e^x-1) / (e^x+1)$, which gives the following equivalent expression for $e^x-1$:

$$
e^x-1 = \tanh\left(\frac{x}{2}\right) (e^x + 1).
$$

The right-hand side avoids catastrophic cancellation for $x \approx 0$, so it produces accurate results even for small x.

Note: because $e^x - 1$ shows up quite often during computation, Julia has a dedicated function `expm1(x)` ("exponential minus 1"), whose specific implementation could be different from the one shown in this solution.

In [3]:
f_old(x) = exp(x) - 1
f_new(x) = tanh(x/2) * (exp(x)+1)

f_new (generic function with 1 method)

### Validation of your implementation

Validate your implementation of `f_new(x)` by using the following code block that compares the numerical derivatives of `f_old(x)` and `f_new(x)` at `x = 0`.  Unlike `f_old(x)` that produces a completely wrong derivative 0.0, `f_new(x)` must produce a derivative close to the exact value 1.0.

In [4]:
x = 0.0
dx = eps() / 10
println("f_old'(0) =  $((f_old(x+dx) - f_old(x)) / dx)")
println("f_new'(0) =  $((f_new(x+dx) - f_new(x)) / dx)")

f_old'(0) =  0.0
f_new'(0) =  1.0


(b) [1 pt] Evaluation of $g(x) = \log(1+x)$ around $x = 0$.  ($\log$ is the natural logarithm.)

- Explain the numerical problem you may experience during this evaluation.  (Hint: Prob. 1(b).)
- Devise a method to avoid this problem.  (Hint: what is the relationship between $e^x-1$ and $\log(1+x)$?)
- Implement your method in `g_new(x)` below.

***Solution.***

When $x$ is much smaller than 1 in magnitude, many mantissa bits are truncated during the evaluation of $1+x$, and thus $\log(1+x)$ is evaluated inaccurately.  

To avoid this problem, note that $e^x - 1$ in Part (a) is the inverse of $\log(1+x)$.  We should be able to find an accurate expression for $y = \log(1+x)$ by inverting the accurate expression for $x = e^y-1$ already found in Part (a) as follows:
$$
x = e^y - 1 = \tanh\left(\frac{y}{2}\right) (e^y + 1) \Longleftrightarrow x = \tanh\left(\frac{y}{2}\right) (x+2) \Longleftrightarrow y = 2\tanh^{-1}\left(\frac{x}{x+2}\right)
$$
for $x>-1$.  Therefore, we have 
$$
\log(1+x) = 2\tanh^{-1}\left(\frac{x}{x+2}\right).
$$

Unlike the left-hand side's argument $1+x$ that is truncated towards a constant value $1$ for small $x$, the right-hand side's argument $\frac{x}{x+2} = x \cdot \frac{1}{x+2}$ has the small $x$ factored out, so its approximation $\frac{x}{x+2} \approx \frac{x}{2}$ for small x still captures variation of small $x$.  Therefore, the right-hand side expression produces accurate results even for small x.

Note: because $\log(1+x)$ shows up quite often during computation, Julia has a dedicated function `log1p(x)` ("log 1 plus"), whose specific implementation could be different from the one shown in this solution.

In [5]:
g_old(x) = log(1+x)
g_new(x) = 2atanh(x/(x+2))

g_new (generic function with 1 method)

### Validation of your implementation

Validate your implementation of `g_new(x)` by using the following code block that compares the numerical derivatives of `g_old(x)` and `g_new(x)` at `x = 0`.  Unlike `g_old(x)` that produces a completely wrong derivative 0.0, `g_new(x)` must produce a derivative close to the exact value 1.0.

In [6]:
x = 0.0
dx = eps() / 10
println("g_old'(0) =  $((g_old(x+dx) - g_old(x)) / dx)")
println("g_new'(0) =  $((g_new(x+dx) - g_new(x)) / dx)")

g_old'(0) =  0.0
g_new'(0) =  1.0


(b) [1 pt] Evaluation of $h(x,y) = \sqrt{x^2+y^2}$.

- Explain at least one numerical problem that is different from those occurred in Parts (a) and (b) that you may experience during this evaluation.
- Devise a method to avoid this problem.
- Implement your method in `h_new(x,y)` below.

***Solution.***

When $x$ or $y$ is sufficiently large, $x^2+y^2$ can suffer from overflow and be evaluated to `Inf`.  Then $\sqrt{x^2+y^2}$ will be evaluated to `Inf` as well, because `sqrt{Inf} = Inf`.  This is undesirable, because in exact arithmetic, the end result $\sqrt{x^2+y^2}$ will have magnitude closer to that of $x$ or $y$ and therefore should be storable in the double-precision floating-point format.

To avoid this problem, we can factor the larger of $x$ and $y$ out of the square root.  For example, if $x > y$, we can transform the equation as
$$
\sqrt{x^2+y^2} = \abs{x} \sqrt{1+(y/x)^2},
$$
and evaluating the transformed equation will not suffer from overflow anymore because $y/x$ has magnitude less than $1$.

Underflow is another situation we have to worry about.  When *both* $x$ and $y$ are too small in magnitude, the direct evaluation of $x^2$ and $y^2$ may suffer from underflow.  This underflow can be avoided similarly by factoring *either* $x$ or $y$ out of the square root, but still you must factor out the greater of the two.  You might be tempted to factor out the smaller of the two, say $x$ when $\abs{x}<\abs{y}$, because then $(y/x)^2$ is greater than $1$ and less likesly to suffer from underflow.  However this route should not be taken because it increases the chance of overflow in $(y/x)^2$.  In fact, it is perfectly fine that $(y/x)^2$ suffers from underflow, because in that case the exact value of $\sqrt{x^2+y^2}$ cannot be stored accurately within the capacity of double precision anyway.

Solutions will receive the full mark if they correctly identify the possibility of either overflow or underflow (or both), even though the validation code block below was created with only the overflow situation in mind.  Points will not be deducted even if you do not take care of the possibility of division by zero occurring for $x = y = 0$.

Note: because $\sqrt{x^2+y^2}$ shows up frequently during computation, Julia has a dedicated function `hypot(x,y)` that avoids overflow and underflow mentioned here.

In [7]:
h_old(x,y) = sqrt(x^2+y^2)
h_new(x,y) = abs(x)>abs(y) ? abs(x)*sqrt(1+(y/x)^2) : (y==0.0 ? 0.0 : abs(y)*sqrt(1+(x/y)^2))

h_new (generic function with 1 method)

### Validation of your implementation

Validate your implementation of `h_new(x,y)` by using the following code block that compares the values of `h_old(x,y)` and `h_new(x,y)`.  The value of `h_old(x,y)` does not make sense here, but the value of your `h_new(x,y)` must.

In [8]:
x = 1e200
y = 1e200
println("h_old(x,y) = $(h_old(x,y))")
println("h_new(x,y) = $(h_new(x,y))")

h_old(x,y) = Inf
h_new(x,y) = 1.414213562373095e200


## Problem 3.  Dual norm and $B$ such that $\left\| A B x\right\| =\left\| A\right\|  \left\| B\right\|  \left\| x\right\|$ [3 pts]

(Partly taken from Exercise 3.6 of Trefethen and Bau)

In this problem, we will prove step-by-step that for a given square matrix $A \in \mathbb{C}^{m\times m}$ and column vector $x \in \mathbb{C}^m$, there exists a square matrix $B$ such that $\left\| A B x\right\| =\left\| A\right\|  \left\| B\right\|  \left\| x\right\|$, where the matrix norm is induced by the vector norm.  (This fact will be used in proving the important Theorem 12.2 of Trefethen & Bau later in the class.)

The *dual norm* turns out to be a useful tool for the proof.  Let $\left\| \cdot \right\|$ be any vector norm on $\mathbb{C}^m$ with $m \ge 2$.  The dual norm $\left\| \cdot \right\|_*$ of the norm $\left\| \cdot \right\|$ is defined as

$$
\left\| x\right\|_* = \max_{v \neq 0} \frac{\left| x^* v\right|}{\left\| v \right\|} = \max_{\left\| v \right\|=1} \left| x^* v \right|.
$$

Note that the dual norm is defined in terms of $\left\| \cdot \right\|$.

Below, the proofs of some parts rely on the proofs of the previous parts.  If some parts are too difficult to prove, you are welcome to take those parts for granted without proof and use them in proving the next parts.

(a) [0.5 pts] Prove that $\left\| \cdot \right\|_*$ is indeed a norm.

***Proof.***

We need to show that $\left\| \cdot \right\|_*$ satisfies the three conditions:
1. *Nonnegativity*.  In the above definition $\left\| x \right\|_*$, $\left| x^* v \right|$ is always nonnegative, so $\left\| x\right\|_* \ge 0$.  We also need to show $\left\| x\right\|_* = 0$ iff $x = 0$, and it is obvious that $\left\| x\right\|_* = 0$ if $x = 0$.  On the other hand, if $x \neq 0$, then 
$$
\begin{align}
\left\| x \right\|_* &= \max_{v\neq 0} \frac{\left| x^* v \right|}{\left\| v\right\|}\\
&\ge \frac{\left| x^* x \right|}{\left\| x\right\|} = \frac{\left\| x \right\|_2^2}{\left\| x \right\|} > 0,
\end{align}
$$
because $\left\| x\right\|_2 > 0$ and $\left\| x \right\| > 0$ for $x \neq 0$.  This means $\left\| x \right\|_* = 0$ only if $x = 0$.

2. *Absolute scalability*.
$$
\begin{aligned}
\left\| \alpha  x \right\|_* &= \max_{\left\| v \right\| =1} \left| (\alpha x)^* v \right|\\
&= \max_{\left\| v \right\| =1} \left| \alpha \right| \left| x^* v \right|\\
&= \left| \alpha \right| \max_{\left\| v \right\| =1} \left| x^* v \right| = \left| \alpha \right| \left\| x \right\|_*.
\end{aligned}
$$

3. *Triangle inequality*.  
$$
\begin{aligned}
\left\| x+y \right\|_* &= \max_{\left\| v \right\| =1} \left| (x+y)^* v \right|\\
&\le \max_{\left\| v \right\| =1} (\left| x^* v \right| + \left| y^* v \right|)\\
&\le \max_{\left\| v \right\| =1} \left| x^* v \right| + \max_{\left\| v \right\| =1}\left| y^* v \right| = \left\| x \right\|_* + \left\| y \right\|_*.
\end{aligned}
$$

(b) [0.5 pts] Prove that $\left\| x\right\|_{**} \leq \left\| x\right\|$ for all $x\in \mathbb{C}^m$, i.e., the dual norm of the dual norm is less than or equal to the original norm.

***Proof.***

For $x = 0$, the inequality holds because $\left\| x\right\|_{**} = \left\| x \right\| = 0$.

For $x \neq 0$, we want to show that $\left\| x \right\|$ is an upper bound of $\left\| x \right\|_{**} = \max_{y \neq 0} \frac{\left| x^* y \right| }{\left\| y \right\|_*}$.  The last expression contains the dual norm $\left\| y \right\|_*$ in the denominator, and from the definition of the dual norm we know $\left\| y \right\|_* \ge \frac{\left| y^* x \right| }{\left\| x\right\| }$ for arbitrary $y$ for the given $x \neq 0$.  This implies $\left\| x \right\| \ge \frac{\left| x^* y \right| }{\left\| y\right\|_*}$ for arbitrary $y \neq 0$.  Using this in the definition of $\left\| x \right\|_{**}$, we get
$$
\begin{align}
\left\| x \right\|_{**} &= \max_{y \neq 0} \frac{\left| x^* y \right| }{\left\| y \right\|_*}\\
&\le \max_{y \neq 0} \left\| x \right\| = \left\| x \right\|.
\end{align}
$$

(c) [0.5 pts] In fact, we have not only $\left\| x\right\|_{**} \leq \left\| x\right\|$ but also $\left\| x\right\|_{**} = \left\| x\right\|$ for all $x\in \mathbb{C}^m$, i.e., the dual norm of the dual norm *is* the original norm.  Using this fact without proof, show that for a given $x\in \mathbb{C}^m$, there exists a nonzero $w\in \mathbb{C}^m$ such that $w^* x = \left\| w\right\|_* \left\| x\right\|$.

***Proof.***

For $x = 0$, the equation holds for any $w$.

For $x \neq 0$, consider the relationship $\left\| x \right\| = \left\| x \right\|_{**} = \max_{y \neq 0} \frac{\left| x^* y \right| }{\left\| y \right\|_*}$.  This means there exists some $u \neq 0$ such that $\left\| x \right\| = \frac{\left| x^* u \right| }{\left\| u \right\|_*}$, or equivalently 
$$
\left| u^* x \right| = \left\| u \right\|_* \left\| x \right\|.
$$

Notice $w = u$ almost satisfies the desired equation for $w$, except that $\left| u^* x \right|$ may not be $u^* x$.  However, even if $u$ is not the desired $w$, we can multiply a phase factor $e^{i\theta}$ to $u$ in order to cancel the nonzero phase angle of $u^* x$ and make $(e^{i\theta} u)^* x$ real and positive without changing the right-hand side of the above equation.  For such phase factor $e^{i\theta}$, we have
$$
(e^{i\theta} u)^* x = \left| (e^{i\theta} u)^* x \right| = \left| u^* x \right| = \left\| u \right\|_* \left\| x \right\| = \left\| e^{i\theta} u \right\|_* \left\| x \right\|,
$$
which means $e^{i\theta} u$ is the desired $w$.

Because $e^{i\theta}$ is chosen to cancel the phase angle in $(e^{i\theta} u)^* x = e^{-i\theta} (u^* x)$,  it must have the same phase angle as $u^* x$.  Therefore, $e^{i\theta} = \frac{u^* x}{\left| u^* x \right|}$, and $$
w = e^{i\theta} u = \frac{u^* x}{\left| u^* x \right|} u.
$$
You can easily check this $w$ indeed satisfies $w^* x = \left\| w\right\|_* \left\| x\right\|$.

(d) [1 pt] Let nonzero $x,y\in \mathbb{C}^m$ be given.  Show that there exists a rank-1 matrix $B = y z^*$ such that $B x = y$ and $\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$, where $\left\| B\right\|$ is the matrix norm of $B$ induced by the vector norm $\left\| \cdot \right\|$.  (Hint: use Part (c).)

***Proof.***

The goal is to find $z$ satisfying the conditions.  The condition $B x = y (z^* x) = y$ for nonzero $y$ implies $z^* x = 1$.  Also, $\left\| B \right\|$ is
$$
\begin{align}
\left\| B \right\| &= \max_{\left\| v \right\| = 1} \left\| B v \right\|\\
&= \max_{\left\| v \right\| = 1} \left\| y (z^* v) \right\|\\
&= \max_{\left\| v \right\| = 1} \left| z^* v \right| \left\| y \right\|\\
&= \left\| y \right\| \max_{\left\| v \right\| = 1} \left| z^* v \right| = \left\| y \right\| \left\| z \right\|_*.
\end{align}
$$
Therefore, the condition $\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$ implies $\left\| z \right\|_* \left\| x \right\| = 1$.

So, we need to find $z$ such that $z^* x = \left\| z \right\|_* \left\| x \right\| = 1$.  In Part (c), we have found $w$ such that $w^* x = \left\| w\right\|_* \left\| x\right\|$ for a given $x$.  By comparison, we can readily construct $z$ by scaling $w$ down by a factor of $\left\| w\right\|_* \left\| x\right\|$, namely $z = \frac{w}{\left\| w\right\|_* \left\| x\right\|}$.

(e) [0.5 pts] For a given square matrix $A\in \mathbb{C}^{m\times m}$ and column vector $x\in \mathbb{C}^m$, show that there exists $B\in \mathbb{C}^{m\times m}$ such that $\left\| A B x \right\| =\left\| A \right\| \left\| B\right\| \left\| x\right\|$, where the matrix norm is induced by the vector norm.  (Hint: use Part (d).)  

***Proof.***

For $x = 0$, any $B$ satisfies the equation.

For $x \neq 0$, note first that there exists a nonzero $y\in \mathbb{C}^m$ such that $\left\| A y \right\| = \left\| A \right\| \left\| y \right\|$ because $\left\| A \right\| = \max_{v \neq 0} \frac{\left\| A v \right\|}{\left\| v \right\|}$.  For such $y \neq 0$ and the given $x \neq 0$, use the result of Part (d) to find $B$ such that $B x = y$ and $\left\| B\right\|  \left\| x\right\| =\left\| y\right\|$.  Then,
$$
\left\| A B x \right\| = \left\| A y \right\| = \left\| A \right\| \left\| y \right\| = \left\| A\right\| \left\| B\right\| \left\| x\right\|,
$$
so we have found a matrix $B$ that satisfies the condition.

## Bonus Problem.  Accurate evaluation of an infinite series [2 pts]

Consider an infinite series $s = \sum_{k=1}^{\infty} \frac{1}{k^4}$.  The goal of this problem is to find the approximate value $\hat{s}$ of $s$ with double-precision accuracy (i.e., $\hat{s} = \mathrm{fl}(s)$).

The terms in the series decrease quickly to 0.  Therefore, the truncated series $s_n = \sum_{k=1}^{n} \frac{1}{k^4}$ would be a good approximation of the infinite series $s$ for sufficiently large $n$.

(a) [1 pt] Find $n$ for which $s_n$ is a double-precision-or-more accurate approximation of $s$.  Use the following hints:
- The error is $s - s_n = \sum_{k=n+1}^{\infty} \frac{1}{k^4}$.  How small must this error be to achieve the desired accuracy in $s_n$?
- What is a close upper bound of the error $s - s_n$?  Can you find one as the area under some curve, which can be calculated by a definite integral?

***Solution.***

Fon $s_n$ to be a double-precision-or-more accurate approximation of $s$, $\abs{s - s_n}$ needs to be less than $\epsmach=\mathrm{eps()}/2$.

From the following figure, we have 
$$
s - s_n = \sum_{k=n+1}^{\infty} \frac{1}{k^4}<\int_{n}^{\infty}\frac{1}{x^4} dx = \frac{1}{3n^3}.
$$

Therefore, to guarantee $s - s_n < \epsmach$, we need to have $1/3n^3 < \epsmach$, or $n > 1/(3 \epsmach)^{1/3}=144263.40\cdots$ equivalently.  Because $n$ is an integer, we conclude that $n$ needs to be at least $144264$.

Of course, $1/3n^3$ in the above argument is a loose upper bound of $s-s_n$, so $s-s_n < \epsmach$ can be achieved for smaller $n$.  However, $n$ different from $144264$ will be accepted as a valid solution only if it is proved that $s - s_n < \epsmach$ for such $n$.

<img src="x⁻⁴.pdf">

(b) [1 pt] In fact, $s = \frac{\pi^4}{90}$ exactly (see [here](https://en.wikipedia.org/wiki/Riemann_zeta_function#Specific_values)), so the desired $\hat{s}$ is nothing but the floating-point representation of $\frac{\pi^4}{90}$. 

Now, the function `s_truncated` below takes `n` as an argument and evaluates the truncated series $s_n$.  You may expect that `s_truncated` would return $\hat{s}$ if $n$ obtained in Part (a) is used as the argument, but you can verify that is not the case using the subsequent code block.

Change one line of `s_truncated` to make it evaluate $s_n$ more accurately.  Briefly explain in words why your change makes `s_truncated` more accurate.  (Hint.  Which one is more accurate: `(2 + eps()) + eps()` or `2 + (eps() + eps())`?)

After this change, you will see that `s_truncated` for $n$ obtained in Part (a) returns $\hat{s}$.

***Solution.***

When two positive double-precision floating-point numbers $x_1 = m_1 \beta^{e_1}$ and $x_2 = m_2 \beta^{e_2}$ with $e_1 > e_2$ are added, they are first added in infinite precision as

$$
x_1 + x_2 = (m_1 + m_2 \beta^{e_2-e_1}) \beta^{e_1},
$$

and the result is approximated by a floating point number.  During this approximation, only the leading 53 bits of the mantissa are kept.  Therefore, many least-significant mantissa bits in $m_2$ are lost.  The greater the difference between $e_1$ and $e_2$ is, the more the mantissa bits are lost.  

If we evaluate $\sum_{k=1}^n 1/k^4$ in the obvious order, for some $m < n$ the partial sum $s_m = \sum_{k=1}^m 1/k^4$ becomes much greater than the next term $1/(m+1)^4$ to add, and therefore many least-significant mantissa bits in $1/(m+1)^4$ are lost during addition.  Though these lost mantissa bits are the least significant bits and therefore constitute a small amount, such loss occurs for all the $\frac{1}{k^4}$ terms for $k > m$ and is accumulated, leading to a noticeable difference in the final sum.

Therefore, to avoid losing the least-significant mantissa bits of the addends, it is important to keep the partial sum as small as possible during summation (so that it is not too much greater than the next term to add), which is achieved by adding the smallest terms first.  For $s_n = \sum_{k=1}^n 1/k^4$, this means we need to add the terms in the reverse order as in the following modified `s_trucated`.

Note that this technique only works when we know the number of terms to add in advance: we cannot reverse the order of addition in the infinite sum $s = \sum_{k=1}^{\infty} 1/k^4$ because it does not have the last term.  This is why we needed to estimate $n$ in Part (a).

In [9]:
function s_truncated(n)
    s = 0.0
    for k = n:-1:1
        s += 1.0/float(k)^4
    end
    
    return s    
end

s_truncated (generic function with 1 method)

In [10]:
n = ceil(1/(3eps()/2)^(1/3))  # use your n obtained in Part (a)
ŝₙ = s_truncated(n)
ŝ = π^4 / 90
ŝ - ŝₙ

0.0